# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract text from a technical PDF
2. Prepare training data for causal language modelling (CLM)
3. Fine-tune a language (llama 3.2 3b) base model using QLoRA
4. Apply the QLoRA and the learned weights to the instruct model
5. Answer some test questions

In [ ]:
# Clone the git repo to access the utilities
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

In [ ]:
# Make sure to be on the repo directory and pull
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git fetch --all
!git reset --hard origin/main

In [ ]:
# Install the required packages from pypi
!pip install -r requirements.txt --quiet

In [ ]:
# Load packages
from transformers import Trainer, TrainingArguments, AutoTokenizer, pipeline
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig, PeftModel
from datasets import Dataset

from mistralpeft.utils import TextExtractor, CLMPreprocessor

In [ ]:
# Login to HuggingFace using Kaggle's secrets to be able to download models
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
login(secret_value_0) 

## 1. Extract Sentences from PDF
Several PDFs from my former research group at university (Thermo-Fluiddynamics Group, Prof. Polifke) are chosen to form the corpus

In [ ]:
# TextExtractor is a simple ETL class to acquire a text corpus
pdf_files = [
    "Dissertation.pdf",
]
    
pdf_urls = [
    "https://mediatum.ub.tum.de/doc/1360567/1360567.pdf",
    "https://mediatum.ub.tum.de/doc/1601190/1601190.pdf",
    "https://mediatum.ub.tum.de/doc/1597610/1597610.pdf"
    "https://mediatum.ub.tum.de/doc/1584750/1584750.pdf",
    "https://mediatum.ub.tum.de/doc/1484812/1484812.pdf",
    "https://mediatum.ub.tum.de/doc/1335646/1335646.pdf",
    "https://mediatum.ub.tum.de/doc/1326486/1326486.pdf",
    "https://mediatum.ub.tum.de/doc/1306410/1306410.pdf",
    "https://mediatum.ub.tum.de/doc/1444929/1444929.pdf",
]

data_path = Path("data/processed_documents.json")
if not data_path.is_file():
    with TextExtractor("data/processed_documents.json") as extractor:
        # Process local files
        extractor.process_documents(pdf_files)
            
        # Process URLs
        extractor.process_documents(pdf_urls, url_list=True)

## 2. Prepare MCLM Training Data

In [ ]:
# Specify the model and load the tokenizer
# Llama 3.2 3B with approx. 3 billion parameters
model_name = "mistralai/Mistral-7B-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Preprocess the corpus for Causal Language Modeling (CLM)
json_file_paths = ["data/processed_documents.json"]
preprocessor = CLMPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

In [ ]:
# Split into training and test set
train_test_set = dataset.train_test_split(test_size=0.1)
print(f"Created {len(train_test_set['train'])} training examples and {len(train_test_set['test'])} test examples")

# Preview a training example
example = train_test_set["train"][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids'][:256]))

## 3. Load and Prepare Model

In [ ]:
# Load the base model
# Q4_K_M quantization of the base model is achieved through BitsAndBytes. It requires CUDA!
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors for Q4_K_M
    )

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config,
    )
if base_model.config.pad_token_id is None:
    base_model.config.pad_token_id = base_model.config.eos_token_id

In [ ]:
# Configure the (Q)LoRA adaptor to use a rank of r=4
model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj",],# "o_proj", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model, config)

## 4. Train the Model

The LoRA adapter has about 6M parameters to train (compared to 3B parameters of the full LLM)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4, # Creates a virtual batch size of 3
    learning_rate=1e-4,
    fp16=True, # numerical precision of adapter is float16
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit", # Memory efficient optimizer
    log_level="info",
    report_to="none",
)

# Initialize trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train(resume_from_checkpoint=False)

In [ ]:
# Save the LoRA adapter weights:
lora_save_path = "lora_weights" 
peft_model.save_pretrained(lora_save_path)

## 6. Test the Model

peft_model = PeftModel.from_pretrained(base_model, "results/checkpoint-231")

In [ ]:
peft_model = PeftModel.from_pretrained(base_model, lora_save_path)

In [ ]:
def test_continuation(model, tokenizer):# Example queries
    queries = [
        "SI is used when processes are either too complex to gain insight using first principles, i.e. physical laws, or the calculation is too costly in terms of time or resources. Its goal is to",
        "In pulsating or oscillating flow, heat transfer can damp, but also drive instabilities.",
        "The unit impulse response is a time domain model. It shows",
    ]
    
    # Generate responses
    for query in queries:
            
        inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True).to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)
        
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"\nQuery: {query}")
        print(f"\nResponse: {text}")
        print("-" * 80)

In [ ]:
### 6.1 Answers to the test questions by the instruct model w/o LoRA

### 6.2 Answers to the test questions by the adapted model

In [ ]:
test_continuation(base_model, tokenizer)

In [ ]:
test_continuation(peft_model, tokenizer)